In [1]:
# %load reviewer2.py
import math
import networkx as nx
import numpy as np
import graspy

from scipy.sparse.linalg import eigsh 
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

###############################################################################
## #####  rpy2 stuff  ######
# import rpy2's package module
import rpy2.robjects.packages as rpackages
import rpy2.robjects as robjects
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

# R package names
packnames = ('mclust')
utils = rpackages.importr('utils')
base = rpackages.importr('base')
utils.chooseCRANmirror(ind=1) # select the first mirror in the list
from rpy2.robjects.vectors import StrVector
utils.install_packages(StrVector(packnames))

mclust = rpackages.importr('mclust')
###############################################################################

/Users/jaewonc78/Envs/TwoTruth-LaLYec9E/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Installing packages into ‘/usr/local/lib/R/3.5/site-library’
(as ‘lib’ is unspecified)

  warnings.warn(x, RRuntimeWarning)


In [3]:
def cluster_km(E):
    return KMeans(n_clusters=2, random_state=0).fit(E).labels_

## renamed the original definition	
def cluster_gmm_original(E):
    tmp =  GaussianMixture(n_components=2).fit(E).predict(E)
    return tmp

## renamed the original definition	
def cluster_gmm_random(E):
    tmp =  GaussianMixture(n_components=2, init_params='random', max_iter=1000).fit(E).predict(E)
    return tmp


###############################################################################
## new def for R's mclust
def cluster_gmm(E):
    nr,nc = E.shape
    Er = robjects.r.matrix(E, nrow=nr, ncol=nc)
    robjects.r.assign("E", Er)
    mc = mclust.Mclust(Er, G=2, verbose=0)
    Y = mc.rx2("classification")

    return np.array(Y, dtype=np.uint8) - 1
###############################################################################


def eval_cluster(c,b):
    # this should be parametric, but I'm lazy.
    # find the smallest cluster (out of two)
    C = set(i for i in range(len(c)) if c[i] == 0)

    for j in range(1,max(c)+1):
        Cp = set(i for i in range(len(c)) if c[i] == j)
        if len(Cp) < len(C):
            C = Cp
        Bs = [ set(i for i in range(len(c)) if b[i] == j) for j in range(max(b)+1)] 
    return [len(C.intersection(B)) for B in Bs]

def cluster_results(E,b):
    return eval_cluster(cluster_gmm(E), b)

def cluster_results_original(E,b):
    return eval_cluster(cluster_gmm_original(E), b)

# mine
def cluster_results_random(E,b):
    return eval_cluster(cluster_gmm_random(E), b)

def experiment(P, method):
    while 1:
        G = nx.generators.stochastic_block_model([int(1000*x) for x in pi],P) 
        if nx.is_connected(G):
            break

    B = nx.get_node_attributes(G,'block') 
    b = np.zeros(len(B),dtype='int')
    for x in range(len(b)):
        b[x] = B[x]

    A = nx.adjacency_matrix(G).astype('d')
    evals_large, evecs_large = eigsh(A, 2, which='LM')
    ASE = np.dot(evecs_large, np.diag(np.sqrt(evals_large)))
    
    L = -nx.normalized_laplacian_matrix(G) 
    for i in range(len(b)):
       L[i,i] += 1.0
    evals_large, evecs_large = eigsh(L, 2, which='LM')
    LSE = np.dot(evecs_large, np.diag(np.sqrt(evals_large)))

    if method=='mclust':
        rval = cluster_results(ASE,b)
        rval.extend(cluster_results(LSE,b)) 
    elif method=='sklearn_original':
        rval = cluster_results_original(ASE,b)
        rval.extend(cluster_results_original(LSE,b)) 
    elif method=='sklearn_random':
        rval = cluster_results_random(ASE,b)
        rval.extend(cluster_results_random(LSE,b)) 

    return rval

# P1 = np.array(
# 	[[0.009, 0.019, 0.000, 0.002],
# 	[0.019, 0.077, 0.002, 0.013],
# 	[0.000, 0.002, 0.009, 0.019],
# 	[0.002, 0.013, 0.019, 0.076]])

P1 = np.array(
    [[0.018932108, 0.042869173, 0.002084243, 0.008436194],
    [0.042869173, 0.112538688, 0.009629583, 0.040333219],
    [0.002084243, 0.009629583, 0.019360040, 0.044269707],
    [0.008436194, 0.040333219, 0.044269707, 0.115217630	]])

pi = np.array([0.28,0.22,0.28,0.22])

In [4]:
import pandas as pd
cols = ['LG', 'LW', 'RG', 'RW'] * 2

In [5]:
# mclust
results = [ experiment(P1, 'mclust') for t in range(10) ] 

df1 = pd.DataFrame(results, columns=cols)
df1

,LG,LW,RG,RW,LG,LW,RG,RW
0,1,220,1,220,279,220,0,0
1,3,219,4,220,280,220,0,0
2,0,220,1,218,280,220,0,0
3,3,220,2,218,280,220,0,0
4,1,220,1,220,280,220,0,0
5,0,218,1,220,279,220,0,0
6,0,218,2,220,280,220,0,0
7,1,220,5,220,279,220,0,0
8,2,219,2,220,279,220,0,0
9,0,220,0,220,279,220,0,0


In [6]:
# original reviewer
results = [ experiment(P1, 'sklearn_original') for t in range(10) ] 

df2 = pd.DataFrame(results, columns=cols)
df2

,LG,LW,RG,RW,LG,LW,RG,RW
0,279,220,0,0,0,0,280,220
1,280,220,0,0,280,220,0,0
2,276,219,0,0,279,220,0,0
3,280,219,1,0,280,220,0,0
4,0,0,280,220,0,0,280,220
5,0,0,279,220,280,220,0,0
6,278,220,0,0,278,220,0,0
7,0,0,279,220,0,0,280,220
8,0,0,279,220,0,0,280,220
9,279,220,0,0,0,0,280,220


In [14]:
# random init
results = [ experiment(P1, 'sklearn_random') for t in range(10) ] 

df3 = pd.DataFrame(results, columns=cols)
df3

,LG,LW,RG,RW,LG,LW,RG,RW
0,0,0,28,220,0,0,0,0
1,0,99,0,202,278,1,0,19
2,2,211,0,198,2,0,128,220
3,0,206,0,191,105,214,0,0
4,0,0,278,220,0,0,3,220
5,0,0,0,214,26,219,0,0
6,0,187,2,216,0,9,0,0
7,0,130,0,88,141,220,12,0
8,0,0,3,216,4,220,83,0
9,0,0,279,220,0,1,35,219
